# Miniproject 2

## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
from typing import (Callable, Iterable, List, Sequence, Tuple, Optional, Dict)

from abc import abstractmethod, ABCMeta
import sys
import math
import random
import dataclasses
from collections import namedtuple

import numpy as np

import scipy.stats
import scipy.special

# Basic data structures.

# x, y coordinate and the rotation of the robot.
Pose = namedtuple("Pose", ["x", "y", "theta"])
# unique id (int) and x, y coordinate of the landmark.
Landmark = namedtuple("Landmark", ["id", "x", "y"])
# r --- distance to the landmark; b --- angle to the landmark, computed by arctan2.
Measurement = namedtuple("Measurement", ["landmark_id", "r", "b"])

Command = namedtuple("Command", ["delta_p", "delta_theta"])


class Inference(metaclass=ABCMeta):
    """Interface for an inference algorithm (localization or SLAM)."""

    @abstractmethod
    def estimated_pose(self) -> Pose:
        ...

    @abstractmethod
    def init(self, init_mean: np.ndarray, init_cov: np.ndarray) -> None:
        ...

    @abstractmethod
    def update(self, measurements: Sequence[Measurement]) -> None:
        ...

    @abstractmethod
    def plot_state(self, ax) -> None:
        ...


@dataclasses.dataclass(frozen=False)
class Simulator:
    # Initial simulator state
    init_mean: Pose = np.zeros(3)
    # 2x2 covariance matrix of the noise of the initial state
    init_covariance: np.ndarray = np.zeros((3, 3))

    # 2x2 covariance matrix of the motion noise.
    motion_noise_covariance: np.ndarray = np.eye(2, dtype=np.float64)
    # 2x2 covariance matrix of the sensor noise.
    sensor_noise_covariance: np.ndarray = np.diag([1, np.deg2rad(5)])**2

    # A set of landmarks
    landmarks: Sequence[Landmark] = (
        Landmark(0, -1, -1),
        Landmark(1, 11, -1),
        Landmark(2, 11, 11),
        Landmark(3, -1, 11),
    )

    # Robot can only sense landmarks within this range
    max_sensing_range: float = np.inf

    # Current state of the robot
    state: Pose = dataclasses.field(init=False, default=None)

    def init(self) -> None:
        # Initialize robot state
        self.state = Pose(*np.random.multivariate_normal(
            self.init_mean, self.init_covariance))

    def simulate_motion(self, command: Command) -> None:
        """Simulate robot motion. This function updates the value of
        self.state.

        Args:
          command: a Command tuple containing fields delta_p(float), the 
          distance of the movement, and delta_theta(float), the rotation of 
          the movement.
        """
        noise = np.random.multivariate_normal(np.zeros(2),
                                              self.motion_noise_covariance)
        x = (self.state.x + math.cos(self.state.theta) *
             (command.delta_p + noise[0]))
        y = (self.state.y + math.sin(self.state.theta) *
             (command.delta_p + noise[0]))
        theta = self.state.theta + command.delta_theta + noise[1]
        self.state = Pose(x, y, theta)

    def simulate_sensing(self) -> Sequence[Measurement]:
        """Simulate the robot sensing process. This function returns a list of
        measurements, in the same order as the self.landmarks list.
        Specifically, it measures the distance and the angle w.r.t. each
        landmark, and add a Gaussian noise on the measurement.

        Returns:
          measurements: a list of measurements.
        """
        measurements = set()
        # Simulate that the measurements orders are random
        for landmark in sorted(self.landmarks, key=lambda _: random.random()):
            r = math.hypot(landmark.x - self.state.x, landmark.y - self.state.y)

            if r >= self.max_sensing_range:
                # Can't sense this landmark, skip
                continue

            b = math.atan2(landmark.y - self.state.y,
                           landmark.x - self.state.x) - self.state.theta

            noise = np.random.multivariate_normal(np.zeros(2),
                                                  self.sensor_noise_covariance)

            measurements.add(
                Measurement(landmark.id, r + noise[0], b + noise[1]))

        return measurements

    def run(
        self,
        commands: Iterable[Command],
        infer: Optional[Inference] = None,
        snapshot_every_n: Optional[int] = 0,
    ) -> Tuple[List[Pose], List[Pose], List[Inference]]:
        """Run the simulation with a sequence of commands. Optionally, execute
        an inference algorithm along way with the simulation.

        Args:
            commands: an interable of commands.
            infer: the inference algorithm to run the simulation with.
            snapshot_every_n: snapshot the inference algorithm every n steps.
                if <= 0, don't take any snapshot. 
        Returns:
            The true poses of the simulation.
            The estimated poses by `infer`, empty if `infer` is None.
            The snapshots of the inference algorithm.
        """
        true_poses, est_poses, snapshots = [], [], []

        self.init()

        true_poses.append(self.state)
        if infer:
            infer.init(self.init_mean, self.init_covariance)
            est_poses.append(infer.estimated_pose())

        for i, command in enumerate(commands):
            # Simulate motion
            self.simulate_motion(command)
            true_poses.append(self.state)
            if infer:
                # Simulate measurement
                measurements = self.simulate_sensing()
                infer.update(command, measurements)
                est_poses.append(infer.estimated_pose())
                if snapshot_every_n > 0 and i % snapshot_every_n == 0:
                    snapshots.append(copy.deepcopy(infer))

        return true_poses, est_poses, snapshots


def implementation_for(cls, method_name: Optional[str] = None):
    """Helper to provide a concrete implementation for a class method.

    This utility allows one to define methods outside of a class defintion.
    Thus allowing one to provide the implementations of a class "incrementally". 
    This utility is useful for gradually submitting method implementations to 
    catsoop. 

    It is intended to be used as a decorator.

    Args:
        cls: a Python class on which we are implementing the method.
        method_name: an optional method name to indicate which method we are 
            implementing. If not provided, infer the method name by using the 
            decorated function's name.

    Example:
        >>> class A: 
        ...   def foo(self):
        ...     raise NotImplementedError()

        >>> @implementation_for(A)
        ... def foo(self):
        ...   print("implemented foo!")

        >>> A().foo()
        implemented foo!
    """

    def decorator(meth: Callable) -> Callable:
        mname = method_name or meth.__name__
        setattr(cls, mname, meth)
        return meth

    return decorator





def drive_in_a_square_commands() -> Iterable[Command]:
    """Commands to simulate the robot's movement in a square."""
    for _ in range(4):
        for i in range(100):
            yield Command(.1, 0)
        yield Command(0, np.pi / 2)


def press(event):
    """matplotlib helper function.

    It processes the keyboard event. If the user hits q, it exits the
    program; otherwise, it closes the current plot.
    """
    import matplotlib.pyplot as plt
    if event.key == 'q':
        sys.exit(0)
    plt.close()


def plot_samples(samples: np.ndarray,
                 xmin: float = -1,
                 xmax: float = 1,
                 ymin: float = -1,
                 ymax: float = 1):
    """matplotlib helper function. It takes a list of samples as input and
    scatter plot the samples.

    Args:
      samples: a numpy array of the samples. This function only plots the first 
        two components: x and y.
      xmin, xmax, ymin, ymax: the range of the x and y axes.
    """
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(autoscale_on=False, aspect='equal')
    ax.axis([xmin, xmax, ymin, ymax])
    fig.canvas.mpl_connect('key_press_event', press)

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.plot(samples[:, 0], samples[:, 1], 'g+')
    plt.show()


def plot_trajectories(true_poses, estimated_poses, fov=((-1, -1), (1, 1))):
    """matplotlib helper function. It overlays both the true pose and the estimated pose.

    Args:
        true_poses: a numpy array of the true robot trajectory. This function 
        only plots the first two components: x and y.
        estimated_poses: a numpy array of the estimated robot trajectory. 
        This function only plots the first two components: x and y.
        xmin, xmax, ymin, ymax: the range of the x and y axes.
    """

    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()
    ax.axis([fov[0][0], fov[1][0], fov[0][1], fov[1][1]])
    fig.canvas.mpl_connect('key_press_event', press)

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.plot([p.x for p in true_poses], [p.y for p in true_poses],
            color=(0, 0, 1, 0.5))
    ax.plot([p.x for p in estimated_poses], [p.y for p in estimated_poses],
            color=(1, 0, 0, 0.5))

    plt.show()


def render_animation(sim: Simulator,
                     true_poses: List[Pose],
                     estimated_poses: List[Pose],
                     snapshots: List[Inference],
                     fov=((-1, -1), (1, 1))):
    import matplotlib.animation as animation
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots()

    # ax.axis([fov[0][0], fov[1][0], fov[0][1], fov[1][1]])

    def update(i):

        ax.clear()

        # Plot landmarks
        ax.scatter([l.x for l in sim.landmarks], [l.y for l in sim.landmarks],
                   marker="x",
                   color="black")
        # Plot trajectory
        tp = true_poses[:i + 1]
        ep = estimated_poses[:i + 1]
        ax.plot([p.x for p in tp], [p.y for p in tp], color=(0, 0, 1, 0.5))
        ax.plot([p.x for p in ep], [p.y for p in ep], color=(1, 0, 0, 0.5))

        snapshots[i].plot_state(ax)

        return ax

    return animation.FuncAnimation(fig, update, len(snapshots))





## Warmup 1


### Question
Let's make sure we know how to sample from a set of values
based on their weights.

Please write a function that takes in a list of samples and associated weights,
and re-samples from the list of samples with replacement according the
likelihood of each sample given by the list of weights.

This should be an easy question --- you should feel free to use the sampling
helper functions in numpy. Specifically, take a look at the function numpy.random.choice.


For reference, our solution is **2** line(s) of code.

In [ ]:
def warmup_1(samples: np.ndarray, weights: np.ndarray, nr_samples: int):
    """Draw N samples from the input list based on their weights.

    Args:
      samples: a numpy array of shape (n_samples, ), a list of samples.
      weights: a numpy float array of shape (n_samples, ), indicating 
        the weights for individual samples.
      nr_samples: an integer, indicating the number of samples to draw.

    Returns:
      resampled_samples: the return numpy array, containing nr_samples
        samples drawn from the input list, based on the weights with replacement.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);
assert np.allclose(warmup_1(np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=np.int64), np.array([0.03191091, 0.10365558, 0.07293406, 0.13085621, 0.12995933, 0.0454193 , 0.04606439, 0.1336077 , 0.15964489, 0.14594763], dtype=np.float64), 10), np.array([6, 8, 7, 6, 4, 7, 4, 9, 9, 4], dtype=np.int64), atol=1e-6)
print('Tests passed.')

## Warmup 2


### Question
We are going to be sampling from Gaussian
distributions in this miniproject. Let us check that we know how to sample from a
multi-variate Gaussian.

Please write a function that takes in a mean vector, a covariance matrix and a
integer number of samples and returns a list of the requested number of
samples drawn from a Gaussian with the given mean and covariance.

This should be an easy question --- you should feel free to use the sampling
helper functions in numpy. Specifically, take a look at the function numpy.random.multivariate_normal.


For reference, our solution is **2** line(s) of code.

In [ ]:
def warmup_2(mean: np.ndarray, covariance: np.ndarray,
             num_particles: int) -> np.ndarray:
    """Sample from a multi-variate Gaussian distribution.

    Args:
      mean: a numpy array of shape (N, ).
      covariance: a numpy array of shape (N, N).
      num_particles: an integer.

    Returns:
      sample: a numpy vector with length num_particles.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);
assert np.allclose(warmup_2(np.array([0.58345008, 0.05778984, 0.13608912], dtype=np.float64), np.array([[1.61405906, 0.93399482, 0.79165733],  [0.93399482, 1.65599579, 1.43616931],  [0.79165733, 1.43616931, 1.25177385]], dtype=np.float64), 5), np.array([[-0.8347847 , -2.30256418, -1.83805061],  [-0.13115218, -3.33917428, -2.91354857],  [-0.47476854, -1.05885092, -0.83435982],  [ 0.29510331, -0.55660856, -0.28675913],  [-0.06919861, -0.94320128, -0.69794801]], dtype=np.float64), atol=1e-6)
print('Tests passed.')

## Experiment 1: Sampling from Multivariate Gaussian


### Question

Please generate a 2D scatter plot of 1000 samples drawn from a 2D Gaussian with `mean = [0, 0]` and
`covariance = [[0.05, 0], [[0, 0.01]]`, using the function you just coded: `warmup_2`.
Consider using the helper function provided: `plot_samples`.

<div class="question question-multiplechoice">
<b>Submission Material 1a:</b> In your submitted pdf, please include the generated figure.
Name this figure as Figure 1a.
</div>

Let's get some intuition about the geometry. Please generate a second 2D scatter plot of
1000 samples drawn from a 2D Gaussian distribution. You should pick the mean and covariance
so that the figure is shown as a ellipse that has a 45-degree angle with the x-y axes.
Specifically, try to modify the mean and covariance to rotate the figure 1a by 45-degree counter-clockwise.

<div class="question question-multiplechoice">
<b>Submission Material 1b:</b> In your submitted pdf, please include the generated figure.
Name this figure as Figure 1b.
</div>

## Experiment 2: Motion Noises


### Question

We have provided a simulator class for you, that simulates the noisy motion
and sensing of the robot. Let's begin by exploring what happens if we don't
actually do any inference, and assume we always know where the robot is.

We have provided a method `drive_in_a_square` that simulates the motion of the
robot driving around the perimeter of a 10 x 10 square. If you call this
method, you will get a list of poses representing the ''true'' trajectory of
the robot.

Please run the method twice, once with the simulator.motion_noise_covariance
set to `zeros((2, 2))` and once with the simulator.motion_noise_covariance set
to `[[.001, 0], [0, np.deg2rad(3)**2]]`. You should see a significant difference
between the trajectory without noise, and the trajectory with noise. If on each step we
knew the noise, the noise wouldn't be a problem. We could just correct for it
as we steer the robot, but the problem is we **don't** know what the noise is
-- it's just the random perturbations added by nature. We have to use the
sensors to know where the robot is and correct for the
''drift'' in the robot motion. **That** is why we perform inference.

**Hint**: Take a look at the helper function `plot_trajectories` to generate the trajectory plot.

<!-- eighth_experiment() -->
<div class="question question-multiplechoice">
<b>Submission Material 2:</b> In your submitted pdf, please include a single figure which has both
experiment results overlapped on each other. Name this figure as Figure 2.
</div>

Now we've convinced ourselves of the need for inference, let us implement
the two functions we need for sampling-based inference.


## Sampling Question 1


### Utilities

Particle Filter for Localization
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:


@dataclasses.dataclass(frozen=True)
class SLAMParticles:
    """A batch of samples for SLAM."""
    # These fields are all arrays of shape (num_particles, )
    x: np.ndarray
    y: np.ndarray
    theta: np.ndarray

    # Mapping from landmark_id to the index of that landmark in
    # `landmarks_x`` and `landmarks_y`
    landmarks_id_to_idx: Dict[int, int]

    landmarks_x: np.ndarray  # shape (num_particles, num_landmarks)
    landmarks_y: np.ndarray  # shape (num_particles, num_landmarks)

    def __post_init__(self):
        assert self.x.shape == self.y.shape == self.theta.shape
        assert (self.landmarks_x.shape == self.landmarks_y.shape)
        assert self.x.shape[0] == self.landmarks_x.shape[0]
        assert self.landmarks_x.shape[1] == len(self.landmarks_id_to_idx)

    def __len__(self) -> int:
        """The number of samples."""
        return self.x.shape[0]


@dataclasses.dataclass
class SLAM(Inference):

    # A 2x2 array for the motion covariance.
    motion_noise_covariance: np.ndarray = np.zeros((2, 2))

    # A 2x2 numpy array for the sensor noise for the
    # measurements of range and bearing to the landmarks
    sensor_noise_covariance: np.ndarray = np.zeros((2, 2))

    # A 2x2 array for the covariance when initializing new landmarks
    new_landmarks_init_covariance: np.ndarray = np.ones((2, 2))

    num_particles: int = 10

    particles: SLAMParticles = None

    def __post_init__(self):
        if self.particles:
            assert len(self.particles) == self.num_particles

    def estimated_pose(self) -> Pose:
        return Pose(
            *map(np.mean,
                 [self.particles.x, self.particles.y, self.particles.theta]))

    def init(self, init_mean: np.ndarray, init_cov: np.ndarray) -> None:
        samples = np.random.multivariate_normal(init_mean,
                                                init_cov,
                                                size=self.num_particles)
        z1 = np.zeros((self.num_particles, 0))
        self.particles = SLAMParticles(
            samples[:, 0],
            samples[:, 1],
            samples[:, 2],
            {},
            z1,
            z1,
        )

    def incorprate_new_landmarks(self,
                                 measurements: Sequence[Measurement]) -> None:
        new_measurements = [
            m for m in measurements
            if m.landmark_id not in self.particles.landmarks_id_to_idx
        ]

        new_landmarks_id_to_idx = dict(self.particles.landmarks_id_to_idx)
        for m in new_measurements:
            new_landmarks_id_to_idx[m.landmark_id] = len(
                new_landmarks_id_to_idx)

        mr = np.array([m.r for m in new_measurements])
        mb = np.array([m.b for m in new_measurements])

        pose = self.estimated_pose()
        est_lx = pose.x + mr * np.cos(mb + pose.theta)
        est_ly = pose.y + mr * np.sin(mb + pose.theta)
        est_landmark_locs = np.stack([est_lx, est_ly], axis=-1)

        # Add sufficiently large noise to initial landmark locations
        sampled_landmark_locs = est_landmark_locs + np.random.multivariate_normal(
            np.zeros(2),
            self.new_landmarks_init_covariance,
            size=(self.num_particles, len(new_measurements)))

        self.particles = dataclasses.replace(
            self.particles,
            landmarks_id_to_idx=new_landmarks_id_to_idx,
            landmarks_x=np.concatenate(
                [self.particles.landmarks_x, sampled_landmark_locs[..., 0]],
                axis=1),
            landmarks_y=np.concatenate(
                [self.particles.landmarks_y, sampled_landmark_locs[..., 1]],
                axis=1),
        )

    # We will incrementally implement methods below using the `implementation_for`
    # helper
    def motion_model(self, command: Command) -> SLAMParticles:
        raise NotImplementedError()

    def compute_weights(self,
                        measurements: Sequence[Measurement]) -> np.ndarray:
        raise NotImplementedError()

    def update(self, command: Command,
               measurements: Sequence[Measurement]) -> SLAMParticles:
        raise NotImplementedError()

    def plot_state(self, ax: 'matplotlib.axes.Axes') -> None:
        ax.quiver(
            self.particles.x,
            self.particles.y,
            np.cos(self.particles.theta),
            np.sin(self.particles.theta),
            angles="xy",
            scale=100,
            color=(1.0, 0, 0, 0.1),
        )
        ax.scatter(self.particles.landmarks_x.flatten(),
                   self.particles.landmarks_y.flatten(),
                   marker="o",
                   color=(0, 1, 0, 0.8),
                   s=0.8)

### Question

Let us now start by building a motion model for a
simple ground robot that moves in straight lines, and performs point turns.

A common motion model for such a robot assumes that the robot has a pose given
by $[x_t, y_t, \theta_t]$ at time $t$. The robot takes a command $[\Delta p_t,
\Delta \theta_t]$ which modifies the robot's pose according to the following:

$$
\begin{bmatrix} x_{t+1} \\ y_{t+1} \\ \theta_{t+1} \end{bmatrix} =
\begin{bmatrix} x_t + \cos(\theta_t) (\Delta p_t + \omega_t) \\
y_t + \sin(\theta_t)(\Delta p_t + \omega_t) \\
\theta_t + \Delta \theta_t + \nu_t
\end{bmatrix}
$$

where $[\omega_t, \nu_t]$ is Gaussian error injected at each time
step, distributed as $\omega_t \sim \mathcal{N}(0, \sigma^2_p)$, and
$\nu_t \sim \mathcal{N}(0, \sigma^2_\theta)$. The units of $\Delta  p_t$ is in
metres and $\Delta \theta_t$ in radians.

Let us assume that the robot's pose at each time step is a random variable,
represented as a set of samples.

Please implement a function that takes in a set of sampled poses
drawn from prior pose random variable $\mathbf{X}_t$
and a control $[\Delta p_t, \Delta \theta_t]$ and returns a set
of samples drawn from the posterior $X_{t+1}$.

**Hint**: Use `np.random.multivariate_normal` to generate the noise.


For reference, our solution is **10** line(s) of code.

In [ ]:
@implementation_for(Localization)
def motion_model(self: Localization, command: Command) -> None:
    """A motion model that simulates a one-step movement of the robot, and
    updates to the new samples.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
            movement, and delta_theta(float), the rotation of the movement.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def sampling_problem1_test():
    # A simulator with no noise
    sim = Simulator(init_covariance=np.zeros((3, 3)),
                    motion_noise_covariance=np.zeros((2, 2)),
                    sensor_noise_covariance=np.zeros((2, 2)))
    sim.init()
    # Particle filter with implemented motion_model
    num_particles = 10
    pf = Localization(
        sim.landmarks,
        motion_noise_covariance=[[.01, 0], [0, np.deg2rad(5)**2]],
        sensor_noise_covariance=sim.sensor_noise_covariance,
        num_particles=num_particles,
    )
    pf.init(sim.init_mean,
            [[.01, 0, 0], [0, .01, 0], [0, 0, np.deg2rad(5)**2]])
    command = Command(1, 0)
    assert pf.motion_model(command) is None, "update should be in-place"
    assert len(pf.particles) == num_particles
    sim.simulate_motion(command)
    pose = sim.state
    est_pose = pf.estimated_pose()
    assert np.allclose(
        [pose.x, pose.y, pose.theta],
        [est_pose.x, est_pose.y, est_pose.theta],
        atol=0.1
    ), "Estimated pose should be quite close to simulation without noise"

sampling_problem1_test()

print('Tests passed.')

## Experiment 3: Update with a Motion Model: 1


### Question

Now that you have implemented these, let's play with the noise
parameters. Let's first use the default motion parameters, of $\sigma^2_p = 1$
and $\sigma^2_\theta = 1$. Let's assume our initial estimate of the robot's
pose is a Gaussian with `mean = [0, 0, 0]` and `covariance = [[.01, 0, 0], [0, .01,
0], [0, 0, np.deg2rad(5)**2]]`. Please sample 1000 samples from this initial estimate and
propagate the samples through the motion model twice, using a control of
$\Delta p = 1m$ and $\Delta \theta = 0$ both times.

Please generate plots of the [x, y] component of the samples after the first and
second motions (two separate plots).
This is pretty noisy motion! You should see the samples much
more spread out than they were at the start.  Why is the distribution long and
skinny after one motion, and then really spread out after the second motion?

<!-- second_experiment() -->
<div class="question question-multiplechoice">
<b>Submission Material 3:</b> In your submitted pdf, please include the generated figures.
Name them as Figure 3a and 3b.
</div>


## Experiment 4: Update with a Motion Model: 2


### Question

Now let's try reducing the noise parameters. Let's set $\sigma^2_p = .01$ and
$\sigma^2_\theta = \text{np.deg2rad}(.5)^2$. Please generate plots of the [x, y]
component of 1000 samples after the same first and second motions as above. The
particle set should be much more compact than before, and track the motion you
commanded.

<!-- third_experiment() -->
<div class="question question-multiplechoice">
<b>Submission Material 4:</b> In your submitted pdf, please include the generated figures.
Name them as Figure 4a and 4b.
</div>


## Experiment 5: Update with a Motion Model: 3


### Question

For both of the two experiments above, we're taking pretty big motions,
commanding a metre at a time. A mobile robot would generally send commands and
get measurements at a much higher frequency. Let's say the robot is sending
commands at 10 Hz, and travelling at 1 m/s. Please implement a method that
moves the robot the same 2m forward, but in .1 m increments. Please upload a
plot of the [x, y] component of the samples after 2 m.

In this question, please use the same reduced noise parameters: $\sigma^2_p = .01$ and
$\sigma^2_\theta = \text{np.deg2rad}(.5)^2$.

<!-- fourth_experiment() -->
<div class="question question-multiplechoice">
<b>Submission Material 5:</b> In your submitted pdf, please include the generated figure.
Name this figure as Figure 5.
</div>


## Experiment 6: Update with a Motion Model: 4


### Question

Even though all we did is changed how often we sent a command to the robot,
the sample set is much more spread out. You might think that the uncertainty
in the robot's position is independent of things like control frequency, but
this plot should show you that they aren't. When we adjust the control
frequency, we also have to adjust the model parameters.

Let's set $\sigma^2_p = .001$. In your submitted pdf, plaese include plots of the 
$[x, y]$ component of 1000 samples after the same first and second motions as 
above. The particle set should be much more compact again, and track the motion 
you commanded.

<!-- fifth_experiment() -->
<div class="question question-multiplechoice">
<b>Submission Material 6:</b> In your submitted pdf, please include the generated figure.
Name this figure as Figure 6.
</div>


## Experiment 7: Update with a Motion Model: 5


### Question

What happens when our motion model has worse rotation noise than translation
noise? Let's leave $\sigma^2_p = .001$ but set $\sigma^2_\theta =
\text{np.deg2rad}(30)^2$. Please generate plots of the $[x, y]$ component of the samples
after 1m, 2m and 3m of motion made in 1m increments. You should see the samples are
tightly contained, but are in a curved shape. This shape is a result of the
non-linear relationship between orientation and position, and is one of the
reasons we often use sampling for localization, rather than a parametric form
such as a Gaussian.

<!-- sixth_experiment() -->
<div class="question question-multiplechoice">
<b>Submission Material 7:</b> In your submitted pdf, please include the generated figures.
Name them as Figure 7a, 7b, and 7c.
</div>



## Experiment 8: Update with a Motion Model: 6


### Question

We still have the relationship between the noise parameters and the control
frequency. Let's leave $\sigma^2_p = .001$ but set $\sigma^2_\theta =
\text{np.deg2rad}(3)^2$. Please generate a plot of the sample distribution after 3m of
motion, but after 30 motions of $.1m$.

<!-- seventh_experiment() -->
<div class="question question-multiplechoice">
<b>Submission Material 8:</b> In your submitted pdf, please include the generated figure.
Name this figure as Figure 8.
</div>


## Sampling Question 2


### Question

Let us now add in a measurement model. Let us assume that
there are four landmarks located at $(x, y)$ locations (-1, -1), (11, -1), (11, 11) and
(-1, 11). The robot is equipped with a sensor that can measure the range and
relative bearing $[r, b]$ from the robot's location to each of these landmarks. If the
robot's pose is $(x, y, \theta)$ and the landmark position is $(l_x, l_y)$,
then range is just the Euclidean distance $$r = \sqrt{(l_x - x)^2 + (l_y
-y)^2}$$, 
and the bearing is the angle to the landmark in the robot's body frame (hence
relative bearing), $$b = atan2(l_y - y, l_x - x) - \theta.$$  Each of
these measurements is corrupted with noise $[q_r, q_b]$ distributed according
to $q_r \sim \mathcal{N}(0, \sigma^2_r)$ and $q_b \sim \mathcal{N}(0,
\sigma^2_b)$.

Please take a look at our implementation in `simulator.simulate_sensing` and
understand how these values are computed.

Now the problem is that for a real robot, even if you know this is the model
that generates the measurements, you can't know the noise that is being
applied. (That would be omniscience!) What we can do is work out how likely
each measurement is.  More precisely, given a set of measurements, we can use
this model to compute the importance weights of a set of samples of the robot
pose. See [Lecture 9 Page 13](https://sicp-s4.mit.edu/_static/fall22/lectures/lec09/L09-Sampling.pdf#page=13)
for details.

Please implement a function that takes a set of samples and measurements and
returns the importance weights of the samples. We encourage you to implement a version
that works for an arbitrary number of landmarks (e.g., more than 4). Although we will
only test your implementation in settings with 4 landmarks.


For reference, our solution is **25** line(s) of code.

In addition to all the utilities defined at the top of the Colab notebook, the following functions are available in this question environment: `motion_model`. You may not need to use all of them.

In [ ]:
@implementation_for(Localization)
def compute_weights(self: Localization,
                    measurements: Sequence[Measurement]) -> np.ndarray:
    """Compute the importance weights of the samples, based on the new
    measurement.

    Args:
        measurements: a sequence of measurements made by the robot.

    Returns:
        weights: a numpy array of importance weights, normalized to 1.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def sampling_problem2_test(compute_weights, pf: Localization,
                           measurements: Sequence[Measurement],
                           result_weights: np.ndarray):
    weights = compute_weights(pf, measurements)
    assert np.allclose(weights, result_weights, atol=1e-4)

sampling_problem2_test(compute_weights, Localization(landmarks=(Landmark(id=0, x=-1, y=-1), Landmark(id=1, x=11, y=-1), Landmark(id=2, x=11, y=11), Landmark(id=3, x=-1, y=11)), motion_noise_covariance=[[0.01, 0], [0, 0.007615435494667714]], sensor_noise_covariance=array([[1.        , 0.        ],
       [0.        , 0.00761544]]), num_particles=100, particles=LocalizationParticles(x=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), y=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), theta=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))), {Measurement(landmark_id=1, r=10.466511352422845, b=-0.11784797459348545), Measurement(landmark_id=0, r=1.4703789046028406, b=-2.4578729951961313), Measurement(landmark_id=2, r=14.871539095163733, b=0.7094067765464278), Measurement(landmark_id=3, r=9.810535196833609, b=1.696567145777771)}, array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01]))
print('Tests passed.')

## Particle Filter Localization


### Question

Now let's put everything together. We've given you a method
`test_localization()` that does the following:
  * Initializes a set of samples drawn from a Gaussian distribution with mean
equal to the robot starting position, and a diagonal covariance matrix equal
to `np.diag(1, 1, np.deg2rad(5)**2)`.
  * Drives the robot along the same pattern as in the `drive_in_a_square` method,
but on each step, after the command motion do the following:
    * Simulates measurements.
    * Update the samples based on the measurements.

What we want you to do is to write a method for step 4: `update` that takes in a
set of samples, the most recent command, the most recent measurement, and the
motion and sensor covariances and does the following:
  * use the motion_model method to update the sample based on the command
  * use compute_weights to compute the sample weights from the
measurement,
  * resample a new set of samples,
  * blur the samples by applying noise drawn from a zero-mean Gaussian with
covariance equal to `np.diag(.001, .001, np.deg2rad(1)**2)`. (You'll need to
recall from lecture why we do this blurring --- if you have machine learning
background, it's a form of regularization, but you don't need to know about
regularization to understand why this is important.)


For reference, our solution is **16** line(s) of code.

In addition to all the utilities defined at the top of the Colab notebook, the following functions are available in this question environment: `compute_weights`, `motion_model`. You may not need to use all of them.

In [ ]:
@implementation_for(Localization)
def update(self: Localization, command: Command,
           measurements: Sequence[Measurement]):
    """Update the samples, based on the command and the new measurement.

    Args:
        samples: a list of samples.
        command: a Command tuple containing fields delta_p(float), the distance of the
        movement, and delta_theta(float), the rotation of the movement.
        measurement: a measurement vector. The measurement is computed by
            `simulator.simulate_sensing` function.

    Returns:
        new_samples: a numpy array of updated samples.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def sampling_problem3_test(_, pf: Localization, command: Command,
                           measurements: Sequence[Measurement], pose: Pose):
    pf.update(command, measurements)
    est_pose = pf.estimated_pose()
    assert np.allclose(
        [pose.x, pose.y, pose.theta],
        [est_pose.x, est_pose.y, est_pose.theta],
        atol=0.3), "Estimated pose should be quite close to our estimation"

sampling_problem3_test(update, Localization(landmarks=(Landmark(id=0, x=-1, y=-1), Landmark(id=1, x=11, y=-1), Landmark(id=2, x=11, y=11), Landmark(id=3, x=-1, y=11)), motion_noise_covariance=array([[0.0001, 0.    ],
       [0.    , 0.0001]]), sensor_noise_covariance=array([[1.00000000e-06, 0.00000000e+00],
       [0.00000000e+00, 9.27917724e-08]]), num_particles=100, particles=LocalizationParticles(x=array([1.00730273, 1.0082067 , 1.00601672, 1.00601641, 1.00651091,
       1.00673879, 1.0056479 , 1.00528662, 1.00664961, 1.00634407,
       1.00717259, 1.00713674, 1.00758623, 1.00662437, 1.0075408 ,
       1.00807182, 1.00709044, 1.00583812, 1.00792024, 1.00518735,
       1.00486222, 1.00609535, 1.00766827, 1.00554004, 1.00659793,
       1.00748429, 1.00681615, 1.00687807, 1.00579081, 1.00570341,
       1.00661133, 1.00859638, 1.00686476, 1.00647296, 1.00582448,
       1.00574882, 1.00697879, 1.00488328, 1.00674228, 1.00657764,
       1.0049294 , 1.00766938, 1.00883084, 1.00719883, 1.00764331,
       1.00626887, 1.00505493, 1.00712798, 1.00601262, 1.00728624,
       1.00750467, 1.00870174, 1.0049673 , 1.00636091, 1.00582382,
       1.00560828, 1.00582004, 1.0057194 , 1.00725904, 1.00625869,
       1.00914811, 1.00849311, 1.00712605, 1.00576135, 1.00726381,
       1.00482082, 1.00674625, 1.00526817, 1.00631799, 1.0057737 ,
       1.00486251, 1.00795701, 1.0058444 , 1.00565331, 1.00666729,
       1.0070602 , 1.00670484, 1.00652485, 1.00422211, 1.00481497,
       1.00506822, 1.00610369, 1.00618893, 1.00638076, 1.00623097,
       1.0065331 , 1.00515001, 1.00592931, 1.00928025, 1.00585582,
       1.00748504, 1.00520833, 1.0081906 , 1.00560902, 1.00628951,
       1.00651723, 1.00715332, 1.00764979, 1.00638846, 1.00836224]), y=array([ 5.88224929e-04, -6.02875336e-05,  1.50115227e-04, -1.13359283e-03,
        9.60042050e-05,  8.50530684e-04,  8.49681370e-05,  1.86668315e-03,
        1.23800694e-03,  6.23453609e-04, -2.54123881e-03, -3.55951493e-04,
        5.58081881e-04,  2.41203964e-04,  1.03298919e-03, -1.38043075e-03,
       -1.49891592e-04, -1.54825432e-03, -1.00393503e-04,  9.13201924e-04,
       -1.06343294e-03,  2.27345950e-04,  5.27597383e-04, -7.96775306e-04,
       -4.46836254e-04, -1.31284967e-03, -2.16343978e-04, -3.44196456e-04,
        6.56390751e-04,  1.65863523e-04, -6.08101257e-04, -1.45190553e-03,
        1.16153380e-03, -1.68451726e-03, -9.46740411e-04,  1.40284474e-03,
        3.07264781e-04,  2.15553054e-04, -8.21534517e-04, -9.57158098e-04,
        5.88742407e-04, -1.00531187e-03,  6.62074289e-04,  8.61768424e-05,
       -8.56149542e-05,  1.20985013e-03,  1.45871475e-03,  1.12737941e-04,
        1.71243545e-04, -1.55799853e-03, -6.75593828e-04,  3.81180002e-05,
        7.29346346e-04, -1.10223019e-03, -9.42093589e-04,  2.46648955e-04,
       -1.36824510e-03, -6.59942705e-04, -2.56316894e-04, -9.71170665e-04,
        1.69596953e-03,  3.55703516e-04, -9.43925064e-05,  8.02051739e-04,
       -1.62958360e-03,  1.96676397e-03,  1.41925505e-03,  8.69006343e-04,
        2.42909141e-05,  4.00885679e-04, -8.02253172e-04,  1.00133102e-03,
        1.80121399e-03, -1.82904981e-04,  8.86864687e-04, -8.57702623e-04,
        1.85934633e-03, -5.84093547e-04,  1.11492456e-03, -3.61113135e-04,
        3.22221644e-06,  2.67383827e-05,  2.72796390e-04,  1.43156776e-03,
       -6.37840556e-04, -3.63437780e-04, -2.77391392e-04, -7.62361537e-04,
       -7.47473178e-04, -7.74295080e-04, -2.98323169e-04,  2.18573582e-05,
        9.24477355e-05, -3.05952575e-04, -9.59255393e-04, -2.00353480e-04,
        4.87015326e-04, -1.36475823e-03, -1.31717479e-03,  6.76342301e-05]), theta=array([0.76011738, 0.76013733, 0.76053628, 0.76035625, 0.76003888,
       0.75979701, 0.75956327, 0.76028336, 0.75973563, 0.76032393,
       0.75975772, 0.76082642, 0.75971238, 0.76039616, 0.75977118,
       0.75988937, 0.75974616, 0.76021467, 0.75993067, 0.76072622,
       0.7598456 , 0.76054435, 0.7598321 , 0.76052419, 0.75999664,
       0.76048361, 0.76018752, 0.75997564, 0.75989056, 0.7604575 ,
       0.76017565, 0.76016904, 0.76035447, 0.76029831, 0.76040243,
       0.76015492, 0.75955353, 0.76022564, 0.76005838, 0.76036149,
       0.76016969, 0.76008351, 0.76008326, 0.76072   , 0.76012948,
       0.75962429, 0.7607085 , 0.76025283, 0.76006447, 0.75989817,
       0.75975002, 0.75987627, 0.76026559, 0.76004514, 0.76002118,
       0.76023782, 0.76052821, 0.76011751, 0.76019298, 0.7604871 ,
       0.76009582, 0.75990723, 0.75975314, 0.75949013, 0.75940647,
       0.75999284, 0.76033719, 0.76010872, 0.76044247, 0.76004968,
       0.76011147, 0.76023454, 0.76011818, 0.76032563, 0.76038882,
       0.76009388, 0.75955721, 0.7601547 , 0.76005681, 0.76023812,
       0.75973049, 0.7601085 , 0.76030179, 0.76049803, 0.75954568,
       0.76028993, 0.76034772, 0.75961186, 0.75965651, 0.75997056,
       0.75977151, 0.76006558, 0.76002246, 0.75991905, 0.7602166 ,
       0.75953049, 0.75979701, 0.76004596, 0.76011041, 0.75991157]))), Command(delta_p=1, delta_theta=0.7853981633974483), {Measurement(landmark_id=2, r=15.5569164763819, b=0.7853303326828786), Measurement(landmark_id=0, r=1.4158616973053026, b=-2.356144463557211), Measurement(landmark_id=1, r=11.044375506449578, b=-0.09110823378307042), Measurement(landmark_id=3, r=11.045399647739101, b=1.6609515497158507)}, Pose(x=1.025000266658993, y=0.000135767690790116, theta=0.7629669345932445))
print('Tests passed.')

## Simulation, Sensing, and Localization


### Question

Let's test this out. Using the same motion noise that we used for the initial
experiments at the beginning of this miniproject that convinced us we needed
inference ($\sigma^2_p = .001, \sigma^2_\theta = \text{np.deg2rad}(3)^2$), and sensor
noise where $\sigma^2_r = 1$ and $\sigma^2_b = \text{np.deg2rad}(5)^2$, please
generate a plot of the vehicle trying to execute the same 10m square
trajectory, but in .1m increments. And also plot the x/y mean of the sample
estimates using measurements simulated after each motion using the
`simulator.simulate_sensing` method we provided.

If everything is working, your estimated and ground truth curves should match
up closely, but not exactly. (You should be very suspicious if they match up
exactly.)

Please take a look at the provided implementation of `test_localization()`.
Use this function to simulate the motion, collect sensory data, and perform localization.
You can use the `plot_trajectories` helpful function to generate the plots.

<!-- ninth_experiment() -->
<div class="question question-multiplechoice">
<b>Submission Material 9:</b> In your submitted pdf, please include the generated figure.
Name this figure as Figure 9.
</div>


In [ ]:


def test_pf_SLAM(simulator: Simulator,
                 num_particles: int = 10) -> Tuple[List[Pose], List[Pose]]:

    return simulator.run(
        drive_in_a_square_commands(),
        SLAM(
            simulator.motion_noise_covariance,
            simulator.sensor_noise_covariance,
            new_landmarks_init_covariance=np.eye(2),
            num_particles=num_particles,
        ),
    )